In [1]:
import ee
import datetime
import ee.mapclient
import pandas as pd
import time
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g6lJ-m1ioPucUZEyFElfCLyvIeTYam3XXlULaSxW3UScmH3DZ8MhZE

Successfully saved authorization token.


In [2]:
##from https://gist.github.com/MareinK/f3dc5aaa52d01ce2c30e0742361392c8


merge = pd.core.reshape.merge
errors = pd.errors
common = pd.core.dtypes.common


def _validate_specification(self):
    super(merge._AsOfMerge, self)._validate_specification()

    # we only allow on to be a single item for on
    # if len(self.left_on) != 1 and not self.left_index:
    #     raise MergeError("can only asof on a key for left")

    # if len(self.right_on) != 1 and not self.right_index:
    #     raise MergeError("can only asof on a key for right")

    # if self.left_index and isinstance(self.left.index, MultiIndex):
    #     raise MergeError("left can only have one index")

    # if self.right_index and isinstance(self.right.index, MultiIndex):
    #     raise MergeError("right can only have one index")

    # set 'by' columns
    if self.by is not None:
        if self.left_by is not None or self.right_by is not None:
            raise errors.MergeError('Can only pass by OR left_by '
                                    'and right_by')
        self.left_by = self.right_by = self.by
    if self.left_by is None and self.right_by is not None:
        raise errors.MergeError('missing left_by')
    if self.left_by is not None and self.right_by is None:
        raise errors.MergeError('missing right_by')

    # add 'by' to our key-list so we can have it in the
    # output as a key
    if self.left_by is not None:
        if not common.is_list_like(self.left_by):
            self.left_by = [self.left_by]
        if not common.is_list_like(self.right_by):
            self.right_by = [self.right_by]

        if len(self.left_by) != len(self.right_by):
            raise errors.MergeError('left_by and right_by must be same length')

        self.left_on = self.left_by + list(self.left_on)
        self.right_on = self.right_by + list(self.right_on)

    # check 'direction' is valid
    if self.direction not in ['backward', 'forward', 'nearest']:
        raise errors.MergeError('direction invalid: {direction}'
                                .format(direction=self.direction))

# perform monkey patch
pd.core.reshape.merge._AsOfMerge._validate_specification = _validate_specification

In [3]:
snodas = ee.ImageCollection("projects/climate-engine/snodas/daily")

In [4]:
multibursts = pd.read_csv("../../movementData/labeledGPS.csv", low_memory = False)
multibursts['acquisition_time'] = pd.to_datetime(multibursts['acquisition_time'])
#filtering to remove elk before snodas was available
multibursts = multibursts[multibursts['acquisition_time']>'2005-1-1']

In [5]:
def getSnowDepth(d):
    
    #getting data from single date
    dateTable = multibursts[multibursts['acquisition_time'].dt.date == d]
    #must sort for merging purposes
    dateTable = dateTable.sort_values(['latitude', 'longitude'])
    #now have a list of geometries of all points at a given date
    points = ee.Geometry.MultiPoint(list(zip(list(dateTable["longitude"]), list(dateTable['latitude']))))
    
    #filtering collection
    snowDepth = snodas.filterDate(d.strftime("%Y-%m-%d")).filterBounds(points)
    
    #extracting depth
    df = pd.DataFrame(ee.ImageCollection(snowDepth.reduce(ee.Reducer.mean())).getRegion(points, 30).getInfo())
    
    #merging
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data without the header row
    df.columns = new_header
    df = df[["longitude", "latitude", "Snow_Depth_mean"]]
    df['longitude'] = df['longitude'].astype(float)
    df['latitude'] = df['latitude'].astype(float)
    df = df.sort_values(['latitude', 'longitude'])
    
    return(pd.merge_asof(dateTable, df))
    

In [6]:
extracted = pd.DataFrame()
days = multibursts['acquisition_time'].dt.date.unique()

In [7]:
t = time.time()
for i in range(0, len(days)):
    try:
        extracted = pd.concat([extracted, getSnowDepth(days[i])])
    except:
        print('day ' + str(i) + ' errored')
    #saving every 50 days
    if (i % 50 == 0):
        print('writing to csv; ' + str(round(i/len(days)*100, 3)) + '% done with total extraction')
        extracted.to_csv("snodasExtraction.csv")
        print("Time so far: " + str(round((time.time() - t)/60, 3)) + " minutes")

#saving when completed as well
extracted.to_csv("snodasExtraction.csv")

writing to csv; 0.0% done with total extraction
Time so far: 0.138 minutes
writing to csv; 0.993% done with total extraction
Time so far: 3.296 minutes
writing to csv; 1.986% done with total extraction
Time so far: 6.233 minutes
writing to csv; 2.979% done with total extraction
Time so far: 9.108 minutes
writing to csv; 3.972% done with total extraction
Time so far: 11.972 minutes
writing to csv; 4.965% done with total extraction
Time so far: 15.171 minutes
writing to csv; 5.958% done with total extraction
Time so far: 18.483 minutes
writing to csv; 6.951% done with total extraction
Time so far: 21.86 minutes
writing to csv; 7.944% done with total extraction
Time so far: 25.305 minutes
writing to csv; 8.937% done with total extraction
Time so far: 28.302 minutes
writing to csv; 9.93% done with total extraction
Time so far: 31.082 minutes
writing to csv; 10.924% done with total extraction
Time so far: 33.906 minutes
writing to csv; 11.917% done with total extraction
Time so far: 36.806 

writing to csv; 100.298% done with total extraction
Time so far: 460.478 minutes
writing to csv; 101.291% done with total extraction
Time so far: 463.893 minutes
writing to csv; 102.284% done with total extraction
Time so far: 467.292 minutes
writing to csv; 103.277% done with total extraction
Time so far: 470.952 minutes
writing to csv; 104.27% done with total extraction
Time so far: 474.818 minutes
